In [37]:
import faiss
import joblib
import numpy as np
import pandas as pd

In [38]:
user_df = pd.read_csv('../data/VL_csv/tn_traveller_master_여행객 Master_E_preprocessed.csv')

In [39]:
user_df.head(3).T

,0,1,2
GENDER,2,2,2
EDU_NM,4,7,6
EDU_FNSH_SE,1.0,1.0,1.0
MARR_STTS,3,2,2
JOB_NM,11,2,2
HOUSE_INCOME,9.0,12.0,5.0
TRAVEL_TERM,2,2,2
TRAVEL_LIKE_SIDO_1,11,42,50
TRAVEL_LIKE_SIDO_2,47,48,42
TRAVEL_LIKE_SIDO_3,41,41,44


In [40]:
# style_cols = [f"TRAVEL_STYL_{i}" for i in range(1, 9)] + ['TRAVELER_ID']
style_cols = [f"TRAVEL_STYL_{i}" for i in range(1, 9)]
style_cols

['TRAVEL_STYL_1',
 'TRAVEL_STYL_2',
 'TRAVEL_STYL_3',
 'TRAVEL_STYL_4',
 'TRAVEL_STYL_5',
 'TRAVEL_STYL_6',
 'TRAVEL_STYL_7',
 'TRAVEL_STYL_8']

In [41]:
style_df = user_df[style_cols]
style_df.head(3).T

,0,1,2
TRAVEL_STYL_1,2,1,3
TRAVEL_STYL_2,4,1,1
TRAVEL_STYL_3,4,1,2
TRAVEL_STYL_4,4,3,5
TRAVEL_STYL_5,4,2,6
TRAVEL_STYL_6,4,1,6
TRAVEL_STYL_7,5,1,1
TRAVEL_STYL_8,5,7,6


In [42]:
style_array = style_df.to_numpy().astype('float32')

# Faiss 거리계산

In [43]:
d = style_array.shape[1]
index = faiss.IndexFlatL2(d)
index.add(style_array)

In [44]:
k = 10
new_user = np.array([[2, 4, 4, 4, 4, 4, 5, 5]], dtype='float32')
D, I = index.search(new_user, k)

In [45]:
print("유사 유저 인덱스:", I[0])
print("유사도 거리:", D[0])
print("유사 유저 스타일:\n", style_df.iloc[I[0]])

유사 유저 인덱스: [   0 1012 1017   37  866 1005 1246 1470 1548 1633]
유사도 거리: [0. 1. 2. 3. 3. 3. 3. 3. 3. 3.]
유사 유저 스타일:
       TRAVEL_STYL_1  TRAVEL_STYL_2  TRAVEL_STYL_3  TRAVEL_STYL_4  \
0                 2              4              4              4   
1012              2              4              4              3   
1017              1              4              4              4   
37                3              3              4              4   
866               2              4              4              3   
1005              2              4              4              4   
1246              1              4              4              5   
1470              3              3              4              3   
1548              2              3              4              4   
1633              1              4              4              4   

      TRAVEL_STYL_5  TRAVEL_STYL_6  TRAVEL_STYL_7  TRAVEL_STYL_8  
0                 4              4              5              5  
10

# 배포용 함수화

In [ ]:
import faiss
import joblib
import numpy as np
import pandas as pd

style_cols = ['TRAVEL_STYL_1',
            'TRAVEL_STYL_2',
            'TRAVEL_STYL_3',
            'TRAVEL_STYL_4',
            'TRAVEL_STYL_5',
            'TRAVEL_STYL_6',
            'TRAVEL_STYL_7',
            'TRAVEL_STYL_8']

def find_nearest_users(input_vec, k=5):
    # 불러와야되는 정보 (pymysql로 df로 불러와야됨)
    user = pd.read_csv("../data/VL_csv/tn_traveller_master_여행객 Master_E.csv")
    travel = pd.read_csv('../data/VL_CSV/tn_travel_여행_E.csv')
    area = pd.read_csv('../data/VL_CSV/tn_visit_area_info_방문지정보_Cleaned_E.csv')
    photo = pd.read_csv('../data/VL_csv/tn_tour_photo_관광사진_E.csv')
    
    
    style_df = user[style_cols]
    style_array = style_df.to_numpy().astype('float32')
    
    input_vec = np.array(input_vec, dtype='float32')
    
    d = style_array.shape[1]
    index = faiss.IndexFlatL2(d)
    index.add(style_array)
    D, I = index.search(input_vec, k)
    
    # 유사 유저의 여행 정보 추출
    similar_users = user.iloc[I[0]]
    travel = travel[travel['TRAVELER_ID'].isin(similar_users['TRAVELER_ID'])]
    travel_ids = travel['TRAVEL_ID'].to_list()
    
    # 여행 정보 바탕으로 이동
    filter_area = ['집', '사무실', '학교', '기타']
    area = area[(area['TRAVEL_ID'].isin(travel_ids)) & (~area['VISIT_AREA_NM'].isin(filter_area))]
    
    cond = photo['VISIT_AREA_ID'].isin(area['VISIT_AREA_ID'].to_list()[:10])

    photo = photo[cond]

    return photo